# **Building and Training a Feedforward Neural Network for Language Modeling**


### Importing The Required Libraries

In [9]:
%%capture
import warnings
from tqdm import tqdm

warnings.simplefilter('ignore')
import time
from collections import OrderedDict

import re

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import string
import time
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from nltk.tokenize import word_tokenize




### Defining Preprocessing Functions 

In [10]:
def preprocess_string(s):
    """
    Preprocesses a given string by performing the following steps:
    
    1. Removes all non-word characters (excluding letters and numbers).
    2. Removes all whitespace characters.
    3. Removes all numeric digits.

    Parameters:
    s (str): The input string to be cleaned.

    Returns:
    str: The processed string with only alphabetic characters, no spaces, and no digits.
    """

    # Remove all non-word characters (everything except letters and numbers)
    # \w matches any word character (letters, numbers, and underscores)
    # \s matches any whitespace characters
    # ^ inside [] negates the selection, so [^\w\s] matches anything that's NOT a word character or whitespace.
    s = re.sub(r"[^\w\s]", '', s)

    # Remove all whitespace characters (spaces, tabs, newlines)
    # \s+ matches one or more whitespace characters.
    s = re.sub(r"\s+", '', s)

    # Remove all digits (0-9)
    # \d matches any digit character.
    s = re.sub(r"\d", '', s)

    return s

In [11]:
def process(word):
    tokens=word_tokenize(word)
    tokens=[preprocess_string(token) for token in tokens]
    tokens=[token.lower() for token in tokens]
    return tokens

In [22]:
def convert_to_indices(tokens):
    vocab = build_vocab_from_iterator([tokens],specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])
    return vocab



### Giving Input 

In [23]:
song= """We are no strangers to love
You know the rules and so do I
A full commitments what Im thinking of
You wouldnt get this from any other guy
I just wanna tell you how Im feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Weve known each other for so long
Your hearts been aching but youre too shy to say it
Inside we both know whats been going on
We know the game and were gonna play it
And if you ask me how Im feeling
Dont tell me youre too blind to see
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Weve known each other for so long
Your hearts been aching but youre too shy to say it
Inside we both know whats been going on
We know the game and were gonna play it
I just wanna tell you how Im feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you"""

### Preprocessing The Input song

In [24]:
tokens=process(song)

In [25]:
vocab=convert_to_indices(tokens)

In [26]:
for token in tokens[0:10]:
    print(vocab[token]) 

22
60
72
76
26
71
3
21
32
74


In [28]:
def get_embedding(vocab):
    embedding_dim=20
    vocab_size=len(vocab)
    return nn.Embedding(vocab_size,embedding_dim)

In [29]:
embeddings=get_embedding(vocab)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    a

In [32]:
index_to_token=vocab.get_itos()
for n in range(2): 
    embedding=embeddings(torch.tensor(n))
    print("word",index_to_token[n])
    print("index",n)
    print( "embedding", embedding)
    print("embedding shape", embedding.shape)

word <unk>
index 0
embedding tensor([-1.3458,  2.1030,  1.5950,  0.3270,  0.1333, -1.2177, -2.6133, -1.1560,
        -0.3544,  2.2936,  0.2522, -1.1585,  1.7332,  0.5333, -1.2225,  1.5741,
        -0.1581,  1.3141, -0.8688,  0.1939], grad_fn=<EmbeddingBackward0>)
embedding shape torch.Size([20])
word na
index 1
embedding tensor([ 1.1005,  2.0894, -1.3037,  0.3833, -1.0886, -1.1893, -1.6744,  1.0692,
        -0.7010,  0.5115, -1.1093, -1.0554,  0.0773, -0.3233, -1.5951,  0.3525,
         2.2583,  0.4809, -0.2596,  1.5278], grad_fn=<EmbeddingBackward0>)
embedding shape torch.Size([20])
